# This notebook expands the TREC COVID dataset with queries from the T5-base finetuned model in the doc2query task

In [1]:
# !pip install transformers -q
# !pip install sentencepiece -q

In [2]:
WORKING_FOLDER="drive/MyDrive/unicamp/ia368v_dd/aula_06"

API_KEYS_FILE="/content/drive/MyDrive/unicamp/ia368v_dd/api_keys_20230324.json"

TREC_COVID_CORPUS_FILENAME="trec_covid_corpus.tsv"
TREC_COVID_EXPANDED_CORPUS_FILENAME="trec_covid_corpus_expanded.tsv"

TREC_COVID_TMP_FILE="tmp_generated_expansion_from_{:06d}_to_{:06d}.pkl"

In [3]:
import os
# from google.colab import drive
import json

import pandas as pd
import numpy as np

from scipy import stats

import pickle

import torch

In [4]:
# drive.mount('/content/drive', force_remount=True)
# os.chdir(WORKING_FOLDER)

In [5]:
from transformers import (AutoTokenizer, 
                          AutoModelForSeq2SeqLM, 
                          Seq2SeqTrainer,
                          Seq2SeqTrainingArguments,

                          GenerationConfig,

                          TrainerCallback, 
                          get_cosine_with_hard_restarts_schedule_with_warmup,
                          DataCollatorForSeq2Seq,

                          )

In [6]:
pd.set_option('display.max_colwidth', None)

## Read the TREC COVID merged data

This data should have been prepared by the `explore_trec_covid.ipynb` notebook.

In [7]:
trec_covid_corpus_df = pd.read_csv(TREC_COVID_CORPUS_FILENAME, sep='\t')

# Prepare T5 model

Load the T5 mode fine-tuned to the doc2query document expansion task.

In [8]:
BEST_FINE_TUNED_T5_MODEL="trained_model/checkpoint-100-19.6221"
BEST_FINE_TUNED_T5_MODEL_MORE_DATA="trained_model_more_data/checkpoint-150-17.8306"
LONGER_FINE_TUNED_T5_MODEL="trained_model/checkpoint-3000"

In [9]:
MODELS_TO_TEST=[BEST_FINE_TUNED_T5_MODEL]

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
instantiated_models = []

for which_model in MODELS_TO_TEST:
    instantiated_models.append(AutoModelForSeq2SeqLM.from_pretrained(which_model).to(device))

In [12]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

/home/eduseiti/.local/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [13]:
tokenizer.model_max_length

512

In [14]:
NUMBER_OF_SEQUENCES=20

In [15]:
generation_params = GenerationConfig(max_new_tokens=200, 
                                     do_sample=True, 
                                     temperature=1.2,
                                     top_k=50,
                                     top_p=0.8,
                                     num_beams=1, 
                                     num_return_sequences=NUMBER_OF_SEQUENCES)

In [16]:
SAVE_AFTER = 1000

In [17]:
START_POSITION=100000
END_POSITION=129186

In [18]:
BATCH_SIZE=1

In [19]:
trec_covid_corpus_no_nan_df = trec_covid_corpus_df.dropna()

In [20]:
document_expansion = []

doc_texts = trec_covid_corpus_no_nan_df.iloc[START_POSITION:END_POSITION]['text'].tolist()
doc_indexes = trec_covid_corpus_no_nan_df.iloc[START_POSITION:END_POSITION].index.tolist()

for i in range(0, END_POSITION - START_POSITION, BATCH_SIZE):
 
    print("Generating topics for sequence={}~{}/{}".format(doc_indexes[i], doc_indexes[i + BATCH_SIZE - 1], trec_covid_corpus_no_nan_df.shape[0]))

    input_ids = tokenizer(doc_texts[i:(i + BATCH_SIZE)], padding=True, return_tensors='pt', truncation=True).input_ids.to(device)

    # print(input_ids.shape)

    generated_text = instantiated_models[0].generate(inputs=input_ids, generation_config=generation_params)

    decoded_text = tokenizer.batch_decode(generated_text, skip_special_tokens=True)

    # print(decoded_text)

    document_expansion += [" ".join(decoded_text[i:i + NUMBER_OF_SEQUENCES]) for i in list(range(0, len(decoded_text), NUMBER_OF_SEQUENCES))]

    if (i > 0) and (i % SAVE_AFTER == 0):
        with open(TREC_COVID_TMP_FILE.format(START_POSITION, END_POSITION), "wb") as outputFile:
            pickle.dump({"doc_indexes": doc_indexes,
                         "doc_expansion": document_expansion}, outputFile, pickle.HIGHEST_PROTOCOL)

Generating topics for sequence=137201~137201/129186
Generating topics for sequence=137202~137202/129186
Generating topics for sequence=137203~137203/129186
Generating topics for sequence=137204~137204/129186
Generating topics for sequence=137205~137205/129186
Generating topics for sequence=137207~137207/129186
Generating topics for sequence=137209~137209/129186
Generating topics for sequence=137210~137210/129186
Generating topics for sequence=137211~137211/129186
Generating topics for sequence=137212~137212/129186
Generating topics for sequence=137213~137213/129186
Generating topics for sequence=137214~137214/129186
Generating topics for sequence=137215~137215/129186
Generating topics for sequence=137216~137216/129186
Generating topics for sequence=137217~137217/129186
Generating topics for sequence=137218~137218/129186
Generating topics for sequence=137219~137219/129186
Generating topics for sequence=137220~137220/129186
Generating topics for sequence=137221~137221/129186
Generating t

Generating topics for sequence=137386~137386/129186
Generating topics for sequence=137388~137388/129186
Generating topics for sequence=137389~137389/129186
Generating topics for sequence=137390~137390/129186
Generating topics for sequence=137391~137391/129186
Generating topics for sequence=137392~137392/129186
Generating topics for sequence=137393~137393/129186
Generating topics for sequence=137394~137394/129186
Generating topics for sequence=137395~137395/129186
Generating topics for sequence=137396~137396/129186
Generating topics for sequence=137397~137397/129186
Generating topics for sequence=137398~137398/129186
Generating topics for sequence=137399~137399/129186
Generating topics for sequence=137400~137400/129186
Generating topics for sequence=137401~137401/129186
Generating topics for sequence=137402~137402/129186
Generating topics for sequence=137403~137403/129186
Generating topics for sequence=137404~137404/129186
Generating topics for sequence=137407~137407/129186
Generating t

Generating topics for sequence=137575~137575/129186
Generating topics for sequence=137576~137576/129186
Generating topics for sequence=137577~137577/129186
Generating topics for sequence=137578~137578/129186
Generating topics for sequence=137579~137579/129186
Generating topics for sequence=137580~137580/129186
Generating topics for sequence=137581~137581/129186
Generating topics for sequence=137582~137582/129186
Generating topics for sequence=137583~137583/129186
Generating topics for sequence=137584~137584/129186
Generating topics for sequence=137585~137585/129186
Generating topics for sequence=137586~137586/129186
Generating topics for sequence=137587~137587/129186
Generating topics for sequence=137588~137588/129186
Generating topics for sequence=137589~137589/129186
Generating topics for sequence=137590~137590/129186
Generating topics for sequence=137591~137591/129186
Generating topics for sequence=137592~137592/129186
Generating topics for sequence=137593~137593/129186
Generating t

Generating topics for sequence=137751~137751/129186
Generating topics for sequence=137752~137752/129186
Generating topics for sequence=137753~137753/129186
Generating topics for sequence=137755~137755/129186
Generating topics for sequence=137758~137758/129186
Generating topics for sequence=137760~137760/129186
Generating topics for sequence=137761~137761/129186
Generating topics for sequence=137764~137764/129186
Generating topics for sequence=137765~137765/129186
Generating topics for sequence=137766~137766/129186
Generating topics for sequence=137767~137767/129186
Generating topics for sequence=137768~137768/129186
Generating topics for sequence=137769~137769/129186
Generating topics for sequence=137770~137770/129186
Generating topics for sequence=137771~137771/129186
Generating topics for sequence=137772~137772/129186
Generating topics for sequence=137773~137773/129186
Generating topics for sequence=137774~137774/129186
Generating topics for sequence=137775~137775/129186
Generating t

Generating topics for sequence=137939~137939/129186
Generating topics for sequence=137940~137940/129186
Generating topics for sequence=137941~137941/129186
Generating topics for sequence=137942~137942/129186
Generating topics for sequence=137943~137943/129186
Generating topics for sequence=137944~137944/129186
Generating topics for sequence=137945~137945/129186
Generating topics for sequence=137946~137946/129186
Generating topics for sequence=137948~137948/129186
Generating topics for sequence=137949~137949/129186
Generating topics for sequence=137950~137950/129186
Generating topics for sequence=137951~137951/129186
Generating topics for sequence=137954~137954/129186
Generating topics for sequence=137956~137956/129186
Generating topics for sequence=137957~137957/129186
Generating topics for sequence=137960~137960/129186
Generating topics for sequence=137961~137961/129186
Generating topics for sequence=137962~137962/129186
Generating topics for sequence=137963~137963/129186
Generating t

...

Generating topics for sequence=170432~170432/129186
Generating topics for sequence=170433~170433/129186
Generating topics for sequence=170434~170434/129186
Generating topics for sequence=170435~170435/129186
Generating topics for sequence=170436~170436/129186
Generating topics for sequence=170437~170437/129186
Generating topics for sequence=170438~170438/129186
Generating topics for sequence=170439~170439/129186
Generating topics for sequence=170440~170440/129186
Generating topics for sequence=170441~170441/129186
Generating topics for sequence=170442~170442/129186
Generating topics for sequence=170443~170443/129186
Generating topics for sequence=170444~170444/129186
Generating topics for sequence=170445~170445/129186
Generating topics for sequence=170446~170446/129186
Generating topics for sequence=170447~170447/129186
Generating topics for sequence=170448~170448/129186
Generating topics for sequence=170450~170450/129186
Generating topics for sequence=170451~170451/129186
Generating t

Generating topics for sequence=170618~170618/129186
Generating topics for sequence=170619~170619/129186
Generating topics for sequence=170620~170620/129186
Generating topics for sequence=170621~170621/129186
Generating topics for sequence=170622~170622/129186
Generating topics for sequence=170623~170623/129186
Generating topics for sequence=170624~170624/129186
Generating topics for sequence=170625~170625/129186
Generating topics for sequence=170626~170626/129186
Generating topics for sequence=170627~170627/129186
Generating topics for sequence=170629~170629/129186
Generating topics for sequence=170630~170630/129186
Generating topics for sequence=170631~170631/129186
Generating topics for sequence=170632~170632/129186
Generating topics for sequence=170633~170633/129186
Generating topics for sequence=170635~170635/129186
Generating topics for sequence=170636~170636/129186
Generating topics for sequence=170637~170637/129186
Generating topics for sequence=170638~170638/129186
Generating t

Generating topics for sequence=170807~170807/129186
Generating topics for sequence=170808~170808/129186
Generating topics for sequence=170810~170810/129186
Generating topics for sequence=170811~170811/129186
Generating topics for sequence=170812~170812/129186
Generating topics for sequence=170813~170813/129186
Generating topics for sequence=170814~170814/129186
Generating topics for sequence=170815~170815/129186
Generating topics for sequence=170816~170816/129186
Generating topics for sequence=170818~170818/129186
Generating topics for sequence=170819~170819/129186
Generating topics for sequence=170820~170820/129186
Generating topics for sequence=170821~170821/129186
Generating topics for sequence=170824~170824/129186
Generating topics for sequence=170825~170825/129186
Generating topics for sequence=170826~170826/129186
Generating topics for sequence=170827~170827/129186
Generating topics for sequence=170828~170828/129186
Generating topics for sequence=170829~170829/129186
Generating t

Generating topics for sequence=171002~171002/129186
Generating topics for sequence=171003~171003/129186
Generating topics for sequence=171004~171004/129186
Generating topics for sequence=171005~171005/129186
Generating topics for sequence=171008~171008/129186
Generating topics for sequence=171009~171009/129186
Generating topics for sequence=171010~171010/129186
Generating topics for sequence=171011~171011/129186
Generating topics for sequence=171012~171012/129186
Generating topics for sequence=171013~171013/129186
Generating topics for sequence=171014~171014/129186
Generating topics for sequence=171015~171015/129186
Generating topics for sequence=171016~171016/129186
Generating topics for sequence=171017~171017/129186
Generating topics for sequence=171018~171018/129186
Generating topics for sequence=171019~171019/129186
Generating topics for sequence=171020~171020/129186
Generating topics for sequence=171021~171021/129186
Generating topics for sequence=171022~171022/129186
Generating t

Generating topics for sequence=171191~171191/129186
Generating topics for sequence=171192~171192/129186
Generating topics for sequence=171193~171193/129186
Generating topics for sequence=171194~171194/129186
Generating topics for sequence=171195~171195/129186
Generating topics for sequence=171197~171197/129186
Generating topics for sequence=171198~171198/129186
Generating topics for sequence=171199~171199/129186
Generating topics for sequence=171200~171200/129186
Generating topics for sequence=171201~171201/129186
Generating topics for sequence=171202~171202/129186
Generating topics for sequence=171203~171203/129186
Generating topics for sequence=171204~171204/129186
Generating topics for sequence=171205~171205/129186
Generating topics for sequence=171206~171206/129186
Generating topics for sequence=171208~171208/129186
Generating topics for sequence=171209~171209/129186
Generating topics for sequence=171210~171210/129186
Generating topics for sequence=171211~171211/129186
Generating t

In [21]:
with open(TREC_COVID_TMP_FILE.format(START_POSITION, END_POSITION), "wb") as outputFile:
    pickle.dump({"doc_indexes": doc_indexes,
                 "doc_expansion": document_expansion}, outputFile, pickle.HIGHEST_PROTOCOL)